# Exploring the MovieLens Dataset with `pySpark`

*Apache Spark* is a popular framework for big data. It supports a wide variety of data analytics tasks including data cleaning, stream processing, and machine learning. It can be used to perform large and parallel computations, performing calculations on a single laptop or a cluster of computers. This makes it a useful tool when the data gets too large for `pandas` to handle. Spark can be accessed in Scala, its native API, as well as in Python, Java, R and SQL. In general the `DataFrame` API in Python should achieve the same performance as in Scala. 

This Jupyter Notebook will demonstrate how to get started using `pySpark` and the `DataFrame` API to perform some basic data analysis, including:
- reading in data
- performing aggregations and joins using the Spark SQL module
- calculating summary statistics

We will use the [MovieLens 20M Dataset](https://grouplens.org/datasets/movielens/) on movie ratings to find out:
- What are the most popular movies?
- What are the top rated movies?
- Which movies are the most polarising?

**Note**: This Notebook assumes that you have pySpark installed and configured to work with the Jupyter Notebook. The purpose of this Notebook is to demonstrate some basic Spark techniques rather than to provide an installation guide. For information on how to get pySpark running on the Jupyter Notebook, please refer to [this blog post](https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f).

## Initialising Spark

To use Spark, we must first initialise a `SparkSession`. This is the entry point to using Spark in an application.

Depending on how we have configured Spark, we may also need to use the `findspark` package to make the `SparkSession` available. 

In [3]:
import findspark
findspark.init()

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MovieLens').getOrCreate()

## Reading in the data

https://grouplens.org/datasets/movielens/

The *MovieLens 20M* Dataset contains 20,000,263 ratings and 465,564 tag applications across 27,278 movies. The dataset was generated in 2016. All users in the dataset rated at least 20 movies. 

The dataset contains six CSV files. We will be using the **`movies`** and **`ratings`** files. Let's see what these two files look like.

To read in a CSV file, we access the `DataFrameReader` class through `read` and then call the `csv()` method on it. We also specify `option("header", "true")` so that the first row of the file is used for the column headers. 

In [5]:
from pyspark.sql.types import IntegerType


# hadoop fs put 
ratings = spark.read.option("header", "true").csv("/dataset/ml-25m/ratings.csv")
ratings = ratings.withColumn("rating", ratings["rating"].cast(IntegerType()))
ratings.show(5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|     5|1147880044|
|     1|    306|     3|1147868817|
|     1|    307|     5|1147868828|
|     1|    665|     5|1147878820|
|     1|    899|     3|1147868510|
+------+-------+------+----------+
only showing top 5 rows



Each row of the `ratings` DataFrame represents one rating for one movie (`movieId`) by one user (`userId`). The ratings use a 5-star scale with half-star increments from 0.5 stars up to 5.0 stars. We can print the DataFrame's column names and types using the `printSchema()` method.

In [4]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)



In [6]:
from pyspark.sql.types import IntegerType


movies = spark.read.option("header", "true").csv("/dataset/ml-25m/movies.csv")
movies = movies.withColumn("movieId", movies["movieId"].cast(IntegerType()))
movies.printSchema()
movies.show(5)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



Each row of the `movies` DataFrame represents one movie and its title and genre(s), indexed by the key `movieId`. We will use this DataFrame to get the movie titles out so we know which movie the ratings in the `ratings` DataFrame are actually referring to. 

## Most popular movies

To get the most popular movies, we are looking for the movies with the highest number of ratings (we use the number of ratings as a proxy for the number of views). To do this, we will perform the following *transformations* on the `ratings` DataFrame: 
- group by `movieId`
- count the number of users (`userId`) associated with each movie 
- rename this column to `num_ratings`
- sort by `num_ratings` in descending order 

In the next cell, we perform these transformations in `pySpark` and store the DataFrame as `most_popular`.

In [5]:
from pyspark.sql.functions import *

most_popular = ratings\
.groupBy("movieId")\
.agg(count("userId"))\
.withColumnRenamed("count(userId)", "num_ratings")\
.sort(desc("num_ratings"))

The DataFrame methods we have used here are:
- `groupBy` - groups the DataFrame by the given column
- `agg` - allows us to perform an aggregate calculation on grouped data (this can be a built-in aggregation function such as *count* or a user defined function)
- `withColumnRenamed` - renames an existing column with a new column name
- `sort` - sorts by the specified column(s)

Because transformations are *lazy* in Spark, the transformations above aren't performed until we call an *action*, such as `show()`, `take()`, or `collect()`.

In [6]:
most_popular.show(10)

+-------+-----------+
|movieId|num_ratings|
+-------+-----------+
|    356|      81491|
|    318|      81482|
|    296|      79672|
|    593|      74127|
|   2571|      72674|
|    260|      68717|
|    480|      64144|
|    527|      60411|
|    110|      59184|
|   2959|      58773|
+-------+-----------+
only showing top 10 rows



This DataFrame contains only the `movieId` and `num_ratings`. The actual title of the movie is stored in the `movies` DataFrame. To get the movie titles, we can join our `most_popular` DataFrame with the `movies` DataFrame on `movieId`. By default, `join` performs an inner join which is what we want in this case.

In [9]:
most_popular_movies = most_popular.join(movies, most_popular.movieId == movies.movieId)
most_popular_movies.show(20, truncate=False)

+-------+-----------+-------+-----------------------------------------+-------------------------------------------+
|movieId|num_ratings|movieId|title                                    |genres                                     |
+-------+-----------+-------+-----------------------------------------+-------------------------------------------+
|296    |79672      |296    |Pulp Fiction (1994)                      |Comedy|Crime|Drama|Thriller                |
|2294   |10937      |2294   |Antz (1998)                              |Adventure|Animation|Children|Comedy|Fantasy|
|48738  |5741       |48738  |Last King of Scotland, The (2006)        |Drama|Thriller                             |
|88140  |8774       |88140  |Captain America: The First Avenger (2011)|Action|Adventure|Sci-Fi|Thriller|War       |
|115713 |12547      |115713 |Ex Machina (2015)                        |Drama|Sci-Fi|Thriller                      |
|1090   |16796      |1090   |Platoon (1986)                           |D

We now have a list of the most popular (or most rated) movies on the *MovieLens* website. As expected, the titles listed here are indeed all well-known movies.

## Top rated movies

We've got the top 10 most popular movies, but now we want to see which movies are perceived to be the best. To get the top rated movies, we are looking for the movies with the highest average rating. To do this, we will use the `ratings` DataFrame and: 

- group by `movieId` 
- calculate the average rating for each movie
- rename this column to `avg_rating`
- sort by `avg_rating` in descending order 

In [10]:
top_rated = ratings\
.groupBy("movieId")\
.agg(avg(col("rating")))\
.withColumnRenamed("avg(rating)", "avg_rating")\
.sort(desc("avg_rating"))

We will again join this DataFrame with the `movies` DataFrame so we know which movie each `movieId` is referring to.

In [11]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId)
top_rated_movies.show(10)

+-------+------------------+-------+--------------------+--------------------+
|movieId|        avg_rating|movieId|               title|              genres|
+-------+------------------+-------+--------------------+--------------------+
|    296|4.0839190681795365|    296| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|   2294|3.0863125171436407|   2294|         Antz (1998)|Adventure|Animati...|
|  48738|3.6237589270161994|  48738|Last King of Scot...|      Drama|Thriller|
|  88140|3.3168452245270115|  88140|Captain America: ...|Action|Adventure|...|
| 115713|3.7710209611859407| 115713|   Ex Machina (2015)|Drama|Sci-Fi|Thri...|
|   1090| 3.778161467015956|   1090|      Platoon (1986)|           Drama|War|
|   3210| 3.525030826140567|   3210|Fast Times at Rid...|Comedy|Drama|Romance|
|   3959|3.5676840215439856|   3959|Time Machine, The...|Action|Adventure|...|
|  27317|          3.379375|  27317|Audition (Ôdishon...|Drama|Horror|Myst...|
|  50802|2.7274305555555554|  50802|Because I Said S

The movies listed here appear to be quite niche. We want to focus on top rated movies that also have a decent number of ratings, so want to take into account both the average rating *and* the number of ratings. We can easily create a DataFrame which has both of these columns by specifying multiple expressions within one `agg()` call. 

In [12]:
top_rated = ratings\
.groupBy("movieId")\
.agg(count("userId"), avg(col("rating")))\
.withColumnRenamed("count(userId)", "num_ratings")\
.withColumnRenamed("avg(rating)", "avg_rating")

In [13]:
top_rated_movies = top_rated.join(movies, top_rated.movieId == movies.movieId).sort(desc("avg_rating"), desc("num_ratings"))
top_rated_movies.show(10)

+-------+-----------+----------+-------+--------------------+--------------------+
|movieId|num_ratings|avg_rating|movieId|               title|              genres|
+-------+-----------+----------+-------+--------------------+--------------------+
| 165787|          3|       5.0| 165787|Lonesome Dove Chu...|             Western|
| 179731|          3|       5.0| 179731|Sound of Christma...|               Drama|
| 148298|          3|       5.0| 148298|       Awaken (2013)|Drama|Romance|Sci-Fi|
| 118268|          3|       5.0| 118268|Borrowed Time (2012)|               Drama|
| 178147|          2|       5.0| 178147|Beatles Stories (...|         Documentary|
| 148114|          2|       5.0| 148114|The Ties That Bin...|  (no genres listed)|
| 169818|          2|       5.0| 169818|FB: Fighting Beat...|              Action|
| 208477|          2|       5.0| 208477|       Kaithi (2019)|     Action|Thriller|
| 179589|          2|       5.0| 179589|  Windstorm 2 (2015)|Adventure|Childre...|
| 16

We see that all of the movies with an average rating of exactly 5.0 have 2 or less ratings. We would like to only consider movies that have achieved some minimum number of ratings. To determine an appropriate threshold, we should investigate the distribution of `num_ratings`. We can do this by calculating some summary statistics within Spark.

In [14]:
# Calculate average, minimum, and maximum of num_ratings
top_rated_movies.select([mean('num_ratings'), min('num_ratings'), max('num_ratings')]).show(1)

+-----------------+----------------+----------------+
| avg(num_ratings)|min(num_ratings)|max(num_ratings)|
+-----------------+----------------+----------------+
|423.3931444442563|               1|           81491|
+-----------------+----------------+----------------+



To calculate quantiles we use the `approxQuantile` method. This method can calculate the quantiles of the specified column approximately or exactly, depending on the value of the relative error parameter. If the relative error parameter is set to 0 then the quantiles are calculated exactly, however this can be expensive. 

In [15]:
# median
top_rated_movies.approxQuantile('num_ratings', [0.5], 0)

[6.0]

In [16]:
# first quartile
top_rated_movies.approxQuantile('num_ratings', [0.25], 0)

[2.0]

In [17]:
# third quartile
top_rated_movies.approxQuantile('num_ratings', [0.75], 0)

[36.0]

The mean is much greater than the median value, suggesting that this distribution is skewed to the right. We will choose a minimum threshold of 500 ratings, however there is no right or wrong answer here and the reader is encouraged to experiment with different values for this threshold.

In [18]:
top_rated_movies.where("num_ratings > 500").show(20, truncate=False)

+-------+-----------+------------------+-------+---------------------------------------------------------------------------+---------------------------+
|movieId|num_ratings|avg_rating        |movieId|title                                                                      |genres                     |
+-------+-----------+------------------+-------+---------------------------------------------------------------------------+---------------------------+
|171011 |1124       |4.483096085409253 |171011 |Planet Earth II (2016)                                                     |Documentary                |
|159817 |1747       |4.464796794504865 |159817 |Planet Earth (2006)                                                        |Documentary                |
|318    |81482      |4.413576004516335 |318    |Shawshank Redemption, The (1994)                                           |Crime|Drama                |
|170705 |1356       |4.398598820058997 |170705 |Band of Brothers (2001)           

We've now gotten a list of the top rated movies on MovieLens, which includes the usual movies considered to be all time greats such as *The Shawshank Redemption* and *Casablanca*. Interestingly, nearly all of these movies appear in the [top 100 of the IMDb top rated movies list](https://www.imdb.com/chart/top) as well, with the exception of the *The Third Man* (listed as #135) and *Band of Brothers* which is technically a TV series rather than a movie.

What's also interesting is that this list of movies is not the same as the list of the most popular movies. *The Shawshank Redemption*, *Schindler's List*, and *The Usual Suspects* were all popular movies which also appear in this list. However, other movies such as *Pulp Fiction*, *Forrest Gump*, and *The Silence of the Lambs* made the top 10 most popular but not the top 10 (or even top 20) most rated. This suggests that some movies actually divide opinion.

## Most polarising movies (Marmite movies)

Next, we will try to answer the question, *What are the most polarising movies*? These are the movies that divide opinon, with people tending to rate them either really highly or really poorly. We will refer to these as *Marmite* movies. Again, we only want to consider movies that achieve some minimum number of ratings - we will stick with our previous threshold of 500 ratings. 

To approach this, we will look for the movies with the highest standard deviation in rating. This is a measure of how much the data varies from the mean, so in this case, how much a movie's ratings vary around its mean rating. A high standard deviation would suggest that the movie's ratings are highly variable. There are other approaches to this as well, for instance, what proportion of the ratings are very positive or very negative.

In [19]:
ratings_stddev = ratings\
.groupBy("movieId")\
.agg(count("userId").alias("num_ratings"), 
     avg(col("rating")).alias("avg_rating"),
     stddev(col("rating")).alias("std_rating")
    )\
.where("num_ratings > 500")

In [19]:
ratings_stddev.show(5)

+-------+-----------+------------------+------------------+
|movieId|num_ratings|        avg_rating|        std_rating|
+-------+-----------+------------------+------------------+
|    296|      67310| 4.174231169217055|0.9760762295742448|
|   1090|      15808| 3.919977226720648|0.8272067263021856|
|   3959|       2869| 3.699372603694667|0.8607671626686735|
|   2294|      10163| 3.303207714257601|0.9047000233824075|
|   6731|       1173|3.5571184995737424|0.9189292350434509|
+-------+-----------+------------------+------------------+
only showing top 5 rows



In [20]:
marmite_movies = ratings_stddev.join(movies, ratings_stddev.movieId == movies.movieId)

In [22]:
marmite_movies.sort(desc("std_rating")).show(15, truncate=False)

+-------+-----------+------------------+------------------+-------+----------------------------------------------------------------------------+-------------------------------------+
|movieId|num_ratings|avg_rating        |std_rating        |movieId|title                                                                       |genres                               |
+-------+-----------+------------------+------------------+-------+----------------------------------------------------------------------------+-------------------------------------+
|27899  |579        |2.8860103626943006|1.4221290413577283|27899  |What the #$*! Do We Know!? (a.k.a. What the Bleep Do We Know!?) (2004)      |Comedy|Documentary|Drama             |
|1924   |2304       |2.6319444444444446|1.4201711823223824|1924   |Plan 9 from Outer Space (1959)                                              |Horror|Sci-Fi                        |
|91104  |516        |2.4728682170542635|1.353614474548174 |91104  |Twilight Saga: Bre

We see that the list of polarising movies includes the *Twilight* movies, the controversial *Passion of the Christ*, and the cult low-budget science fiction movie *Plan 9 from Outer Space*.

## One-hot 

[Spark API](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.OneHotEncoder.html)

In [8]:
# 为了能够让 python 找到 pyspark，使用 findspark
import findspark
findspark.init()

# 为了使用 RDDs，创建 SparkSession
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# 创建 SparkConf 和 SparkSession
conf=SparkConf()\
        .setMaster('local[*]')\
        .setAppName("Spark Read Hive")\
        .setExecutorEnv("spark.executor.memory","4g")\
        .setExecutorEnv("spark.driver.memory","4g")

spark=SparkSession.builder\
        .config(conf=conf)\
        .enableHiveSupport()\
        .getOrCreate()

In [10]:
movies.head(10)

[Row(movieId=1, title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId=2, title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId=3, title='Grumpier Old Men (1995)', genres='Comedy|Romance'),
 Row(movieId=4, title='Waiting to Exhale (1995)', genres='Comedy|Drama|Romance'),
 Row(movieId=5, title='Father of the Bride Part II (1995)', genres='Comedy'),
 Row(movieId=6, title='Heat (1995)', genres='Action|Crime|Thriller'),
 Row(movieId=7, title='Sabrina (1995)', genres='Comedy|Romance'),
 Row(movieId=8, title='Tom and Huck (1995)', genres='Adventure|Children'),
 Row(movieId=9, title='Sudden Death (1995)', genres='Action'),
 Row(movieId=10, title='GoldenEye (1995)', genres='Action|Adventure|Thriller')]

In [11]:
from pyspark.ml.feature import OneHotEncoder


encoder = OneHotEncoder(inputCols=["movieId"],
                        outputCols=["movieIdVector"])
model = encoder.fit(movies)
encoded = model.transform(movies)
encoded.show(10)

+-------+--------------------+--------------------+-------------------+
|movieId|               title|              genres|      movieIdVector|
+-------+--------------------+--------------------+-------------------+
|      1|    Toy Story (1995)|Adventure|Animati...| (209171,[1],[1.0])|
|      2|      Jumanji (1995)|Adventure|Childre...| (209171,[2],[1.0])|
|      3|Grumpier Old Men ...|      Comedy|Romance| (209171,[3],[1.0])|
|      4|Waiting to Exhale...|Comedy|Drama|Romance| (209171,[4],[1.0])|
|      5|Father of the Bri...|              Comedy| (209171,[5],[1.0])|
|      6|         Heat (1995)|Action|Crime|Thri...| (209171,[6],[1.0])|
|      7|      Sabrina (1995)|      Comedy|Romance| (209171,[7],[1.0])|
|      8| Tom and Huck (1995)|  Adventure|Children| (209171,[8],[1.0])|
|      9| Sudden Death (1995)|              Action| (209171,[9],[1.0])|
|     10|    GoldenEye (1995)|Action|Adventure|...|(209171,[10],[1.0])|
+-------+--------------------+--------------------+-------------

## 数值型特征的处理 - 分桶

[分桶 API](https://spark.apache.org/docs/latest/ml-features#quantilediscretizer)

![5675f0777bd9275b5cdd8aa166cebd4e.jpeg](assets/5675f0777bd9275b5cdd8aa166cebd4e.jpeg)

In [12]:
from pyspark.sql.functions import *


# 利用打分表ratings计算电影的平均分、被打分次数等数值型特征
ratings_features = ratings.groupBy(col("movieId")).agg(count(lit(1)), avg(col("rating")), variance(col("rating")))
ratings_features = ratings_features.withColumnRenamed('count(1)', 'ratingCount').withColumnRenamed('avg(rating)', 'avgRating').withColumnRenamed('var_samp(rating)', 'ratingVar')

In [15]:
from pyspark.ml.feature import QuantileDiscretizer


# 分桶处理，创建QuantileDiscretizer进行分桶，将打分次数这一特征分到100个桶中
discretizer = QuantileDiscretizer(numBuckets=100, inputCol="ratingCount", outputCol="ratingCountBucket")

In [16]:
from pyspark.ml.feature import MinMaxScaler


# 归一化处理，创建MinMaxScaler进行归一化，将平均得分进行归一化
scaler = MinMaxScaler(inputCol="avgRating", outputCol="scaledFeatures")

In [17]:
discretizer.fit(ratings_features).transform(ratings_features).show(10)

+-------+-----------+------------------+------------------+-----------------+
|movieId|ratingCount|         avgRating|         ratingVar|ratingCountBucket|
+-------+-----------+------------------+------------------+-----------------+
|    296|      79672|4.0839190681795365|1.0065760077085952|             48.0|
|   2294|      10937|3.0863125171436407|1.0113863372180323|             48.0|
|  48738|       5741|3.6237589270161994|0.6037141606890231|             47.0|
|  88140|       8774|3.3168452245270115|1.1377146102604472|             47.0|
| 115713|      12547|3.7710209611859407|0.8280841082007967|             48.0|
|   1090|      16796| 3.778161467015956|0.7719815198631416|             48.0|
|   3210|       8110| 3.525030826140567|0.9390083539639243|             47.0|
|   3959|       2785|3.5676840215439856| 0.828840359891867|             45.0|
|  27317|       1600|          3.379375|1.0060784083802372|             44.0|
|  50802|        576|2.7274305555555554| 1.436881038647343|     

[Spark MLib 文档](https://spark.apache.org/docs/latest/ml-features#minmaxscaler)

## Conclusion

This tutorial has demonstrated how to use the `pySpark DataFrame` API to perform some simple data analysis tasks. In particular, we have seen how to perform aggregations, joins, and compute summary statistics on large datasets. There is a lot more that could be done with this dataset, including investigating other ways to identify polarising movies, looking at the effect of movie genres, and building a recommender system. Note that when working in `pySpark`, it may useful to refer back to the [official pySpark documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html). 